In [ ]:
%load_ext autoreload
%autoreload 2
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import sys
sys.path.append('./datasets')
sys.path.append('./models')
sys.path.append('./utils')
sys.path.append('./callbacks')
sys.path.append('./visualization')

from utils.experiment_utils import CSVLogger, run_experiment, load_dataset, load_dataset_folds, run_cross_validation, load_dataset_train_test, run_experiment_n_times

from callbacks.LatentSpacePlotter import LatentSpacePlotter
from callbacks.LatentSpaceEvolutionPlotter import LatentSpaceEvolutionPlotter
from callbacks.LatentSpaceEvolutionGIFPlotter import LatentSpaceEvolutionGIFPlotter

from visualization.metric_plots import plot_ROC_curve

from models.AE import AE
from models.BAE import BAE
from models.SAE import SAE
from models.CAE import CAE
from models.KSAE import KSAE
from models.CAEAA import CAEAA

In [ ]:
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median').set_output(transform='pandas')),
    ('scaler', StandardScaler().set_output(transform='pandas'))
])

In [ ]:
KDDCUP99 = load_dataset_train_test("KDDCUP99", "../data/KDDCUP99/preprocessed/", pipeline=pipeline)
# CICIDS2017 = load_dataset_train_test("CICIDS2017", "../data/CIC-IDS2017/preprocessed/", pipeline=pipeline)
UNSWNB15 = load_dataset_train_test("UNSW-NB15", "../data/UNSW-NB15/preprocessed/", pipeline=pipeline)
CTU13_08 = load_dataset_train_test("CTU-13_08", "../data/CTU-13/preprocessed/08", pipeline=pipeline)
CTU13_09 = load_dataset_train_test("CTU-13_09", "../data/CTU-13/preprocessed/09", pipeline=pipeline)
CTU13_10 = load_dataset_train_test("CTU-13_10", "../data/CTU-13/preprocessed/10", pipeline=pipeline)
CTU13_13 = load_dataset_train_test("CTU-13_13", "../data/CTU-13/preprocessed/13", pipeline=pipeline)

In [ ]:
common_params = {
    'batch_norm': True,
    'optimizer': "Adadelta",
    'optimizer_params': {'rho': 0.9},
    'scheduler': "StepLR",
    'scheduler_params': {'step_size':30, 'gamma':0.5},
}

num_epochs = 100
latent_evolution_epochs = [0, 15, 99]
n_runs = 5

series_name = "defend"

In [ ]:
# AE

variants = [
    {
        'dataset': CICIDS2017,
        'params': 
        {
            'input_size': CICIDS2017['train'].x.shape[1],
            'hidden_sizes': [1024, 512, 9],
            'initial_lr': 1e-1,
            'threshold_quantile': 0.95,
        }
    },
    # {
    #     'dataset': KDDCUP99,
    #     'params': 
    #     {
    #         'input_size': KDDCUP99['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-1,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': UNSWNB15,
    #     'params': 
    #     {
    #         'input_size': UNSWNB15['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-1,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_08,
    #     'params': 
    #     {
    #         'input_size': CTU13_08['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-1,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_09,
    #     'params': 
    #     {
    #         'input_size': CTU13_09['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 8],
    #         'initial_lr': 1e-1,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_10,
    #     'params': 
    #     {
    #         'input_size': CTU13_10['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-1,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_13,
    #     'params': 
    #     {
    #         'input_size': CTU13_13['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 7],
    #         'initial_lr': 1e-1,
    #         'threshold_quantile': 0.95,
    #     }
    # },
]

logger = CSVLogger(f"../out/logs/{series_name}/AE.csv")

for variant in variants:
    model = AE(**common_params, **variant['params'])
    records = run_experiment_n_times(
        model=model,
        dataset=variant['dataset'],
        max_epochs=num_epochs,
        trainer_callbacks=[
            LatentSpacePlotter(
                f"../out/figures/{series_name}/latent_space/AE",
                "latent_space",
                variant['dataset']['train'].name,
                plot_epochs=[num_epochs - 1]
            ),
            LatentSpaceEvolutionPlotter(
                f"../out/figures/{series_name}/latent_space_evolution/AE",
                "latent_space_evolution",
                variant['dataset']['train'].name,
                plot_epochs=latent_evolution_epochs
            )
        ],
        experiment_name=f"AE_{series_name}_test_{variant['dataset']['train'].name}",
        run_name=f"run",
        validation_set="test",
        n_runs=n_runs,
        roc_curve_plot=f"../out/figures/{series_name}/roc_curves/AE/roc_curve_{variant['dataset']['train'].name}.png"
    )

    logger.log(records)

In [ ]:
# CAE

variants = [
    #  {
    #     'dataset': CICIDS2017,
    #     'params': 
    #     {
    #         'input_size': CICIDS2017['train'].x.shape[1],
    #         'hidden_sizes': [1024, 512, 9],
    #         'initial_lr': 1e-1,
    #         'num_clusters': 2,
    #         'clustering_force': 300,
    #         'centering_force': 1500,
    #         'threshold_quantile': 0.99,
    #     }
    # },
    # {
    #     'dataset': KDDCUP99,
    #     'params': 
    #     {
    #         'input_size': KDDCUP99['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-1,
    #         'num_clusters': 2,
    #         'clustering_force': 300,
    #         'centering_force': 1500,
    #         'threshold_quantile': 0.99,
    #     }
    # },
    {
        'dataset': UNSWNB15,
        'params': 
        {
            'input_size': UNSWNB15['train'].x.shape[1],
            'hidden_sizes': [256, 64, 9],
            'initial_lr': 1e-1,
            'num_clusters': 4,
            'clustering_force': 1500,
            'centering_force': 300,
            'threshold_quantile': 0.95,
        }
    },
    # {
    #     'dataset': CTU13_08,
    #     'params': 
    #     {
    #         'input_size': CTU13_08['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-1,
    #         'num_clusters': 2,
    #         'clustering_force': 800,
    #         'centering_force': 800,
    #         'threshold_quantile': 0.99,
    #     }
    # },
    # {
    #     'dataset': CTU13_09,
    #     'params': 
    #     {
    #         'input_size': CTU13_09['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 8],
    #         'initial_lr': 1e-1,
    #         'num_clusters': 2,
    #         'clustering_force': 300,
    #         'centering_force': 1500,
    #         'threshold_quantile': 0.98,
    #     }
    # },
    # {
    #     'dataset': CTU13_10,
    #     'params': 
    #     {
    #         'input_size': CTU13_10['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-1,
    #         'num_clusters': 2,
    #         'clustering_force': 300,
    #         'centering_force': 1500,
    #         'threshold_quantile': 0.99,
    #     }
    # },
    # {
    #     'dataset': CTU13_13,
    #     'params': 
    #     {
    #         'input_size': CTU13_13['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 7],
    #         'initial_lr': 1e-1,
    #         'num_clusters': 2,
    #         'clustering_force': 300,
    #         'centering_force': 1500,
    #         'threshold_quantile': 0.99,
    #     }
    # },
]

logger = CSVLogger(f"../out/logs/{series_name}/CAE.csv")

for variant in variants:
    model = CAE(**common_params, **variant['params'])
    records = run_experiment_n_times(
        model=model,
        dataset=variant['dataset'],
        max_epochs=num_epochs,
        trainer_callbacks=[
            LatentSpacePlotter(
                f"../out/figures/{series_name}/latent_space/CAE",
                "latent_space",
                variant['dataset']['train'].name,
                plot_epochs=[num_epochs - 1]
            ),
            LatentSpaceEvolutionPlotter(
                f"../out/figures/{series_name}/latent_space_evolution/CAE",
                "latent_space_evolution",
                variant['dataset']['train'].name,
                plot_epochs=latent_evolution_epochs
            ),
            LatentSpaceEvolutionGIFPlotter(
                f"../out/figures/{series_name}/latent_space_evolution_gif/CAE",
                "latent_space_evolution_gif",
                variant['dataset']['train'].name,
                plot_interval=1
            )
        ],
        experiment_name=f"CAE_{series_name}_{variant['dataset']['train'].name}",
        run_name=f"run",
        validation_set="test",
        n_runs=n_runs,
        roc_curve_plot=f"../out/figures/{series_name}/roc_curves/CAE/roc_curve_{variant['dataset']['train'].name}.png"
    )

    logger.log(records)

In [ ]:
# SAE cen

variants = [
    {
        'dataset': CICIDS2017,
        'params': 
        {
            'input_size': CICIDS2017['train'].x.shape[1],
            'hidden_sizes': [256, 64, 9],
            'initial_lr': 5e-3,
            'lmb': 50,
            'occ_algorithm':'centroid',
            'occ_fit_sample_size': 15000,
            'fit_occ_once': True,
            'threshold_quantile': 0.95,
        }
    },
    # {
    #     'dataset': KDDCUP99,
    #     'params': 
    #     {
    #         'input_size': KDDCUP99['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 5e-3,
    #         'lmb': 50,
    #         'occ_algorithm':'centroid',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': UNSWNB15,
    #     'params': 
    #     {
    #         'input_size': UNSWNB15['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'centroid',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_08,
    #     'params': 
    #     {
    #         'input_size': CTU13_08['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'centroid',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_09,
    #     'params': 
    #     {
    #         'input_size': CTU13_09['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 8],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'centroid',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_10,
    #     'params': 
    #     {
    #         'input_size': CTU13_10['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'centroid',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_13,
    #     'params': 
    #     {
    #         'input_size': CTU13_13['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 7],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'centroid',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'threshold_quantile': 0.95,
    #     }
    # },
]

logger = CSVLogger(f"../out/logs/final/SAE_cen_2.csv")

for variant in variants:
    model = SAE(**common_params, **variant['params'])
    records = run_experiment_n_times(
        model=model,
        dataset=variant['dataset'],
        max_epochs=50,
        trainer_callbacks=[
            LatentSpacePlotter(
                "../out/figures/final/latent_space/SAE_CEN",
                "latent_space",
                variant['dataset']['train'].name,
                plot_epochs=[49]
            ),
            LatentSpaceEvolutionPlotter(
                "../out/figures/final/latent_space_evolution/SAE_CEN",
                "latent_space_evolution",
                variant['dataset']['train'].name,
                plot_epochs=[0, 15, 49]
            )
        ],
        experiment_name=f"SAE_cen_final_{variant['dataset']['train'].name}",
        run_name=f"run",
        validation_set="test",
        n_runs=n_runs,
        roc_curve_plot=f"../out/figures/final/roc_curves/SAE_CEN/roc_curve_{variant['dataset']['train'].name}.png"
    )

    logger.log(records)

In [ ]:
# SAE lof

variants = [
    {
        'dataset': CICIDS2017,
        'params': 
        {
            'input_size': CICIDS2017['train'].x.shape[1],
            'hidden_sizes': [256, 64, 9],
            'initial_lr': 5e-3,
            'lmb': 50,
            'occ_algorithm':'lof',
            'occ_fit_sample_size': 15000,
            'fit_occ_once': True,
            'n_neighbors' : 200,
            'threshold_quantile': 0.95,
        }
    },
    # {
    #     'dataset': KDDCUP99,
    #     'params': 
    #     {
    #         'input_size': KDDCUP99['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 5e-3,
    #         'lmb': 50,
    #         'occ_algorithm':'lof',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'n_neighbors' : 200,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': UNSWNB15,
    #     'params': 
    #     {
    #         'input_size': UNSWNB15['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 30,
    #         'occ_algorithm':'lof',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'n_neighbors' : 200,
    #     }
    # },
    # {
    #     'dataset': CTU13_08,
    #     'params': 
    #     {
    #         'input_size': CTU13_08['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'lof',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'n_neighbors' : 200,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_09,
    #     'params': 
    #     {
    #         'input_size': CTU13_09['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 8],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'lof',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'n_neighbors' : 200,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_10,
    #     'params': 
    #     {
    #         'input_size': CTU13_10['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 10,
    #         'occ_algorithm':'lof',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'n_neighbors' : 200,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_13,
    #     'params': 
    #     {
    #         'input_size': CTU13_13['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 7],
    #         'initial_lr': 1e-2,
    #         'lmb': 30,
    #         'occ_algorithm':'lof',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'n_neighbors' : 200,
    #         'threshold_quantile': 0.95,
    #     }
    # },
]

logger = CSVLogger(f"../out/logs/final/SAE_lof.csv")

for variant in variants:
    model = SAE(**common_params, **variant['params'])
    records = run_experiment_n_times(
        model=model,
        dataset=variant['dataset'],
        max_epochs=50,
        trainer_callbacks=[
            LatentSpacePlotter(
                "../out/figures/final/latent_space/SAE_LOF",
                "latent_space",
                variant['dataset']['train'].name,
                plot_epochs=[49]
            ),
            LatentSpaceEvolutionPlotter(
                "../out/figures/final/latent_space_evolution/SAE_LOF",
                "latent_space_evolution",
                variant['dataset']['train'].name,
                plot_epochs=[0, 15, 49]
            )
        ],
        experiment_name=f"SAE_lof_final_{variant['dataset']['train'].name}",
        run_name=f"run",
        validation_set="test",
        n_runs=5,
        roc_curve_plot=f"../out/figures/final/roc_curves/SAE_LOF/roc_curve_{variant['dataset']['train'].name}.png"
    )

    logger.log(records)

In [ ]:
# SAE svm

variants = [
    {
        'dataset': CICIDS2017,
        'params': 
        {
            'input_size': CICIDS2017['train'].x.shape[1],
            'hidden_sizes': [256, 64, 9],
            'initial_lr': 1e-2,
            'lmb': 50,
            'occ_algorithm':'svm',
            'occ_fit_sample_size': 15000,
            'fit_occ_once': True,
            'nu' : 0.05,
            'threshold_quantile': 0.95,
        }
    },
    # {
    #     'dataset': KDDCUP99,
    #     'params': 
    #     {
    #         'input_size': KDDCUP99['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'svm',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'nu' : 0.05,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': UNSWNB15,
    #     'params': 
    #     {
    #         'input_size': UNSWNB15['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'svm',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'nu' : 0.05,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_08,
    #     'params': 
    #     {
    #         'input_size': CTU13_08['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'svm',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'nu' : 0.05,
    #         'threshold_quantile': 0.99,
    #     }
    # },
    # {
    #     'dataset': CTU13_09,
    #     'params': 
    #     {
    #         'input_size': CTU13_09['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 8],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'svm',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'nu' : 0.05,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_10,
    #     'params': 
    #     {
    #         'input_size': CTU13_10['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'svm',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'nu' : 0.05,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_13,
    #     'params': 
    #     {
    #         'input_size': CTU13_13['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 7],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'svm',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'nu' : 0.05,
    #         'threshold_quantile': 0.95,
    #     }
    # },
]

logger = CSVLogger(f"../out/logs/final/SAE_svm_2.csv")

for variant in variants:
    model = SAE(**common_params, **variant['params'])
    records = run_experiment_n_times(
        model=model,
        dataset=variant['dataset'],
        max_epochs=50,
        trainer_callbacks=[
            LatentSpacePlotter(
                "../out/figures/final/latent_space/SAE_OCSVM",
                "latent_space",
                variant['dataset']['train'].name,
                plot_epochs=[49]
            ),
            LatentSpaceEvolutionPlotter(
                "../out/figures/final/latent_space_evolution/SAE_OCSVM",
                "latent_space_evolution",
                variant['dataset']['train'].name,
                plot_epochs=[0, 15, 49]
            )
        ],
        experiment_name=f"SAE_svm_final_{variant['dataset']['train'].name}",
        run_name=f"run",
        validation_set="test",
        n_runs=5,
        roc_curve_plot=f"../out/figures/final/roc_curves/SAE_OCSVM/roc_curve_{variant['dataset']['train'].name}.png"
    )

    logger.log(records)

In [ ]:
# KSAE SEP cen

variants = [
    # {
    #     'dataset': CICIDS2017,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 3,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': CICIDS2017['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'centroid',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'threshold_quantile': 0.99,
    #     }
    # },
    # {
    #     'dataset': KDDCUP99,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 3,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': KDDCUP99['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'centroid',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'threshold_quantile': 0.99,
    #     }
    # },
    # {
    #     'dataset': UNSWNB15,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 3,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': UNSWNB15['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 30,
    #         'occ_algorithm':'centroid',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_08,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 2,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': CTU13_08['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 30,
    #         'occ_algorithm':'centroid',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'threshold_quantile': 0.99,
    #     }
    # },
    {
        'dataset': CTU13_09,
        'params': 
        {
            'kmeans_n_clusters' : 3,
            'kmeans_fit_quantile': 0.9,
            'kmeans_fit_sample_size': 15000,
            'base_model': SAE,
            'input_size': CTU13_09['train'].x.shape[1],
            'hidden_sizes': [256, 64, 8],
            'initial_lr': 1e-2,
            'lmb': 30,
            'occ_algorithm':'centroid',
            'occ_fit_sample_size': 15000,
            'fit_occ_once': True,
            'threshold_quantile': 0.98,
        }
    },
    # {
    #     'dataset': CTU13_10,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 4,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': CTU13_10['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 30,
    #         'occ_algorithm':'centroid',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'threshold_quantile': 0.98,
    #     }
    # },
    # {
    #     'dataset': CTU13_13,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 2,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': CTU13_13['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 7],
    #         'initial_lr': 1e-2,
    #         'lmb': 30,
    #         'occ_algorithm':'centroid',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'threshold_quantile': 0.98,
    #     }
    # },
]

logger = CSVLogger(f"../out/logs/final/KSAE_SEP_cen_2.csv")

for variant in variants:
    model = KSAE(**common_params, **variant['params'])
    records = run_experiment_n_times(
        model=model,
        dataset=variant['dataset'],
        max_epochs=num_epochs,
        experiment_name=f"KSAE_cen_final_2_{variant['dataset']['train'].name}",
        run_name=f"run",
        validation_set="test",
        n_runs=n_runs,
        roc_curve_plot=f"../out/figures/final/roc_curves/KSAE_cen/roc_curve_{variant['dataset']['train'].name}.png"
    )

    logger.log(records)

In [ ]:
# KSAE SEP svm

variants = [
    # {
    #     'dataset': CICIDS2017,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 2,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': CICIDS2017['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'svm',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'nu' : 0.05,
    #         'threshold_quantile': 0.99,
    #     }
    # },
    # {
    #     'dataset': KDDCUP99,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 2,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': KDDCUP99['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'svm',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'nu' : 0.05,
    #         'threshold_quantile': 0.99,
    #     }
    # },
    # {
    #     'dataset': UNSWNB15,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 3,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': UNSWNB15['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'svm',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'nu' : 0.05,
    #         'threshold_quantile': 0.95,
    #     }
    # },
    # {
    #     'dataset': CTU13_08,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 2,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': CTU13_08['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'svm',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'nu' : 0.05,
    #         'threshold_quantile': 0.99,
    #     }
    # },
    {
        'dataset': CTU13_09,
        'params': 
        {
            'kmeans_n_clusters' : 3,
            'kmeans_fit_quantile': 0.9,
            'kmeans_fit_sample_size': 15000,
            'base_model': SAE,
            'input_size': CTU13_09['train'].x.shape[1],
            'hidden_sizes': [256, 64, 8],
            'initial_lr': 1e-2,
            'lmb': 50,
            'occ_algorithm':'svm',
            'occ_fit_sample_size': 15000,
            'fit_occ_once': True,
            'nu' : 0.05,
            'threshold_quantile': 0.95,
        }
    },
    # {
    #     'dataset': CTU13_10,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 4,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': CTU13_10['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'svm',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'nu' : 0.05,
    #         'threshold_quantile': 0.98,
    #     }
    # },
    # {
    #     'dataset': CTU13_13,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 2,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': CTU13_13['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 7],
    #         'initial_lr': 1e-2,
    #         'lmb': 50,
    #         'occ_algorithm':'svm',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'nu' : 0.05,
    #         'threshold_quantile': 0.98,
    #     }
    # },
]

logger = CSVLogger(f"../out/logs/final/KSAE_SEP_svm_2.csv")

for variant in variants:
    model = KSAE(**common_params, **variant['params'])
    records = run_experiment_n_times(
        model=model,
        dataset=variant['dataset'],
        max_epochs=num_epochs,
        experiment_name=f"KSAE_svm_final_{variant['dataset']['train'].name}",
        run_name=f"run",
        validation_set="test",
        n_runs=5,
        roc_curve_plot=f"../out/figures/final/roc_curves/KSAE_svm/roc_curve_{variant['dataset']['train'].name}.png"
    )

    logger.log(records)

In [ ]:
# KSAE SEP lof

variants = [
    #  {
    #     'dataset': CICIDS2017,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 2,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': CICIDS2017['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 30,
    #         'occ_algorithm':'lof',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'n_neighbors' : 200,
    #     }
    # },
    # {
    #     'dataset': KDDCUP99,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 2,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': KDDCUP99['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 30,
    #         'occ_algorithm':'lof',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'n_neighbors' : 200,
    #     }
    # },
    # {
    #     'dataset': UNSWNB15,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 2,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': UNSWNB15['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 30,
    #         'occ_algorithm':'lof',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'n_neighbors' : 200,
    #     }
    # },
    # {
    #     'dataset': CTU13_08,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 2,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': CTU13_08['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 30,
    #         'occ_algorithm':'lof',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'n_neighbors' : 200,
    #     }
    # },
    # {
    #     'dataset': CTU13_09,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 3,
    #         'kmeans_fit_quantile': 0.95,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': CTU13_09['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 8],
    #         'initial_lr': 1e-2,
    #         'lmb': 30,
    #         'occ_algorithm':'lof',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'n_neighbors' : 200,
    #     }
    # },
    # {
    #     'dataset': CTU13_10,
    #     'params': 
    #     {
    #         'kmeans_n_clusters' : 2,
    #         'kmeans_fit_quantile': 0.9,
    #         'kmeans_fit_sample_size': 15000,
    #         'base_model': SAE,
    #         'input_size': CTU13_10['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-2,
    #         'lmb': 30,
    #         'occ_algorithm':'lof',
    #         'occ_fit_sample_size': 15000,
    #         'fit_occ_once': True,
    #         'n_neighbors' : 200,
    #     }
    # },
    {
        'dataset': CTU13_13,
        'params': 
        {
            'kmeans_n_clusters' : 2,
            'kmeans_fit_quantile': 0.9,
            'kmeans_fit_sample_size': 15000,
            'base_model': SAE,
            'input_size': CTU13_13['train'].x.shape[1],
            'hidden_sizes': [256, 64, 7],
            'initial_lr': 1e-2,
            'lmb': 30,
            'occ_algorithm':'lof',
            'occ_fit_sample_size': 15000,
            'fit_occ_once': True,
            'n_neighbors' : 200,
        }
    },
]

logger = CSVLogger(f"../out/logs/final/KSAE_SEP_lof_2.csv")

for variant in variants:
    model = KSAE(**common_params, **variant['params'])
    records = run_experiment_n_times(
        model=model,
        dataset=variant['dataset'],
        max_epochs=num_epochs,
        experiment_name=f"KSAE_lof_final_{variant['dataset']['train'].name}",
        run_name=f"run",
        validation_set="test",
        n_runs=n_runs,
        roc_curve_plot=f"../out/figures/final/roc_curves/KSAE_lof/roc_curve_{variant['dataset']['train'].name}.png"
    )

    logger.log(records)

In [ ]:
# BAE

variants = [
    # {
    #     'dataset': CICIDS2017,
    #     'params': 
    #     {
    #         'birch_threshold': 0.05,
    #         'birch_branching_factor':50,
    #         'birch_n_clusters' : 2,
    #         'birch_fit_quantile': 0.9,
    #         'birch_fit_sample_size': 15000,
    #         'base_model': AE,
    #         'input_size': CICIDS2017['train'].x.shape[1],
    #         'hidden_sizes': [256, 64, 9],
    #         'initial_lr': 1e-1,
    #         'threshold_quantile': 0.99,
    #     }
    # },
    {
        'dataset': KDDCUP99,
        'params': 
        {
            'birch_threshold': 0.05,
            'birch_branching_factor':50,
            'birch_n_clusters' : 2,
            'birch_fit_quantile': 0.9,
            'birch_fit_sample_size': 15000,
            'base_model': AE,
            'input_size': KDDCUP99['train'].x.shape[1],
            'hidden_sizes': [256, 64, 9],
            'initial_lr': 1e-1,
            'threshold_quantile': 0.95,
        }
    },
    {
        'dataset': UNSWNB15,
        'params': 
        {
            'birch_threshold': 0.05,
            'birch_branching_factor':50,
            'birch_n_clusters' : 4,
            'birch_fit_quantile': 0.9,
            'birch_fit_sample_size': 15000,
            'base_model': AE,
            'input_size': UNSWNB15['train'].x.shape[1],
            'hidden_sizes': [256, 64, 9],
            'initial_lr': 1e-1,
            'threshold_quantile': 0.95,
        }
    },
    {
        'dataset': CTU13_08,
        'params': 
        {
            'birch_threshold': 0.05,
            'birch_branching_factor':50,
            'birch_n_clusters' : 3,
            'birch_fit_quantile': 0.9,
            'birch_fit_sample_size': 15000,
            'base_model': AE,
            'input_size': CTU13_08['train'].x.shape[1],
            'hidden_sizes': [256, 64, 9],
            'initial_lr': 1e-1,
            'threshold_quantile': 0.95,
        }
    },
    {
        'dataset': CTU13_09,
        'params': 
        {
            'birch_threshold': 0.05,
            'birch_branching_factor':50,
            'birch_n_clusters' : 2,
            'birch_fit_quantile': 0.9,
            'birch_fit_sample_size': 15000,
            'base_model': AE,
            'input_size': CTU13_09['train'].x.shape[1],
            'hidden_sizes': [256, 64, 8],
            'initial_lr': 1e-1,
            'threshold_quantile': 0.95,
        }
    },
    {
        'dataset': CTU13_10,
        'params': 
        {
            'birch_threshold': 0.05,
            'birch_branching_factor':50,
            'birch_n_clusters' : 2,
            'birch_fit_quantile': 0.9,
            'birch_fit_sample_size': 15000,
            'base_model': AE,
            'input_size': CTU13_10['train'].x.shape[1],
            'hidden_sizes': [256, 64, 9],
            'initial_lr': 1e-1,
            'threshold_quantile': 0.95,
        }
    },
    {
        'dataset': CTU13_13,
        'params': 
        {
            'birch_threshold': 0.05,
            'birch_branching_factor':50,
            'birch_n_clusters' : 2,
            'birch_fit_quantile': 0.9,
            'birch_fit_sample_size': 15000,
            'base_model': AE,
            'input_size': CTU13_13['train'].x.shape[1],
            'hidden_sizes': [256, 64, 7],
            'initial_lr': 1e-1,
            'threshold_quantile': 0.95,
        }
    },
]

logger = CSVLogger(f"../out/logs/final/BAE_2.csv")

for variant in variants:
    model = BAE(**common_params, **variant['params'])
    records = run_experiment_n_times(
        model=model,
        dataset=variant['dataset'],
        max_epochs=num_epochs,
        experiment_name=f"BAE_final_{variant['dataset']['train'].name}",
        run_name=f"run",
        validation_set="test",
        n_runs=n_runs,
        roc_curve_plot=f"../out/figures/final/roc_curves/BAE/roc_curve_{variant['dataset']['train'].name}.png"
    )

    logger.log(records)